In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import urllib
import re
from bs4 import BeautifulSoup

In [2]:
document = urllib.request.urlopen("https://www.cpubenchmark.net/multi_cpu.html").read()
soup = BeautifulSoup(document)

In [4]:
MIN_PASSMARK_SCORE = 18_000

def parse_row(row):
    name, score, price = row.find_all("td")
    name = name.get_text().strip()
    passmark = score.find("span")
    score = score.get_text().strip()
    price = price.get_text().strip()
    
    if not score or score == "NA":
        score = None
    else:
        score = float(score)
        
    if score:
        mouseover = passmark['onmouseover']
        passmark = int(re.match('^pp_STAD\(event,(\d+),', mouseover)[1])
    else:
        passmark = None
        
    return name, score, passmark, price
        
chart = soup.select("div#value table.chart")[0]
rows = chart.select("tr:nth-child(n+3)")
rows = map(parse_row, rows)
# Filter out rows without a price/performance score
rows = filter(lambda row: row[1] is not None, rows)
# Sort by score
rows = list(sorted(rows, key=lambda row: row[1] , reverse=True))
# Filter out rows with a passmark score lower than MIN_PASSMARK_SCORE
rows = filter(lambda row: row[2] > MIN_PASSMARK_SCORE, rows)

for name, score, passmark, price in rows:
    print("{}: {} - {} ({})".format(name, score, passmark, price))

[Dual CPU] Intel Xeon E5-4650 @ 2.70GHz: 98.86 - 18880 ($190.98*)
[Dual CPU] Intel Xeon E5-2680 @ 2.70GHz: 81.41 - 18209 ($223.68)
[Dual CPU] Intel Xeon E5-2643 v2 @ 3.50GHz: 63.62 - 21039 ($330.68)
[Dual CPU] Intel Xeon E5-2690 @ 2.90GHz: 61.69 - 19618 ($318.00)
[Dual CPU] Intel Xeon E5-2670 v2 @ 2.50GHz: 59.41 - 20911 ($351.98)
[Dual CPU] Intel Xeon E5-2680 v2 @ 2.80GHz: 56.42 - 21438 ($379.98)
[Quad CPU] Intel Xeon E5-4650 @ 2.70GHz: 55.95 - 21372 ($381.96*)
[Dual CPU] Intel Xeon E5-2670 @ 2.60GHz: 53.17 - 18172 ($341.74)
[Dual CPU] Intel Xeon E5-2660 v2 @ 2.20GHz: 48.03 - 18156 ($378.00)
[Dual CPU] Intel Xeon E5-2690 v2 @ 3.00GHz: 43.35 - 23615 ($544.80)
[Dual CPU] Intel Xeon E5-2650 v2 @ 2.60GHz: 38.1 - 18974 ($498.00)
[Dual CPU] Intel Xeon E5-2650 v3 @ 2.30GHz: 28.28 - 20362 ($719.98*)
[Dual CPU] Intel Xeon E5-2687W @ 3.10GHz: 25.19 - 20905 ($830.00*)
[Dual CPU] Intel Xeon E5-2697 v3 @ 2.60GHz: 19.21 - 28246 ($1,470.00)
[Dual CPU] Intel Xeon E5-2470 v2 @ 2.40GHz: 18.24 - 18572 ($